# DeepLabV3+
> 3D implementation of DeepLabV3 

In [ ]:
# hide
import sys
sys.path.append("..")
from nbdev.showdoc import *

In [ ]:
# default_exp models.deeplab
# export 
from fastai.basics import *
from fastai.vision.all import create_body, hook_outputs
from fastai.vision.models.unet import _get_sz_change_idxs
from faimed3d.basics import *
from faimed3d.layers import *
from faimed3d.models.unet import ResizeToOrig

In [ ]:
# export
class SequentialExForDeepLab(Module):
    """Like `nn.Sequential`, but with ModuleList semantics, and can access module input. 
    Also supports hybrid classification and segmentation models"""
    def __init__(self, *layers): self.layers = nn.ModuleList(layers)

    def forward(self, x):
        res = x
        for l in self.layers:
            res.orig = x
            nres = l(res)
            # We have to remove res.orig to avoid hanging refs and therefore memory leaks
            res.orig, nres.orig = None, None
            res = nres
        if hasattr(res, 'class_pred'): 
            class_pred = res.class_pred
            res.class_pred = None
            return res, class_pred
        return res

    def __getitem__(self,i): return self.layers[i]
    def append(self,l):      return self.layers.append(l)
    def extend(self,l):      return self.layers.extend(l)
    def insert(self,i,l):    return self.layers.insert(i,l)

## DeepLabV3+

Implementation of DeepLabV3+ for 3D. Translates the 2D version from https://github.com/giovanniguidi/deeplabV3-PyTorch to 3D. Adds the functionality to allow mulitple encoders, similar to DynamicUnet. However, works probably best with larger encoders, such as ResNet50. 

### ASPP


In [ ]:
# export
class ASPPPooling(nn.Sequential):
    "Pooling Layer for ASPP"
    def __init__(self, ni, nf, norm_type=None, act_cls=defaults.activation):
        super(ASPPPooling, self).__init__(
            nn.AdaptiveAvgPool3d(1),
            ConvLayer(ni=ni, nf=nf, ks=1, ndim=3, bias=False, norm_type=None, act_cls=act_cls)
        )

    def forward(self, x):
        size = x.shape[-3:]
        for module in self:
            x = module(x)
        return F.interpolate(x, size=size, mode='trilinear', align_corners=False)


class ASPP(SequentialExForDeepLab):
    "3D Atrous Spatial Pyramid Pooling"
    def __init__(self, ni, dilations, nf, norm_type=None, act_cls=defaults.activation, ps=0.5):
        conv_layers = [ConvLayer(ni=ni, nf=nf, ks=1, bias=False, ndim=3, norm_type=norm_type, act_cls=act_cls)]
        dilations = tuple(dilations)
        for dilation in dilations:
            conv_layers.append(ConvLayer(ni=ni, nf=nf, ndim=3, ks=3, dilation=dilation, padding=dilation, 
                                         norm_type=norm_type, act_cls=act_cls))
        pooling = ASPPPooling(ni=ni, nf=nf, norm_type=norm_type, act_cls=act_cls)
        self.layers = nn.ModuleList([*conv_layers, pooling])
        self.project = nn.Sequential(
            ConvLayer(ni=len(self.layers)*nf, nf=nf, ks=1, bias=False, ndim=3, 
            norm_type=norm_type, act_cls=act_cls),
            nn.Dropout(ps))

    def forward(self, x):
        res = [module(x) for module in self.layers]
        return self.project(torch.cat(res, dim=1))

In [ ]:
ASPP(ni=2048, dilations=[1, 6, 12, 18], nf=256, norm_type=NormType.Batch)(torch.randn(10, 2048, 1, 3, 3)).size()

torch.Size([10, 256, 1, 3, 3])

In [ ]:
ASPP(ni=2048, dilations=[1, 12, 24, 36], nf=256, norm_type=NormType.Batch)(torch.randn(10, 2048, 1, 3, 3)).size()

torch.Size([10, 256, 1, 3, 3])

In [ ]:
# export
class DeepLabDecoder(Module):
    "Decoder Block for DynamicDeeplab"
    def __init__(self, ni, low_lvl_ni, hook, n_out, norm_type=None, 
                 act_cls=defaults.activation, ps=0.5):
        self.hook = hook
        
        self.low_lvl_conv = ConvLayer(low_lvl_ni, low_lvl_ni//2, ks=1, ndim=3, bias=False, 
                                      norm_type=norm_type, act_cls=act_cls)

        self.last_conv = nn.Sequential(
                ConvLayer(ni+low_lvl_ni//2, ni, ks=3, ndim=3, stride=1, padding=1, bias=False, 
                          norm_type=norm_type, act_cls=act_cls), 
                nn.Dropout(ps),
                ConvLayer(ni, ni, ks=3, ndim=3, stride=1, padding=1, bias=False, 
                          norm_type=norm_type, act_cls=act_cls), 
                nn.Dropout(ps/5),
                nn.Conv3d(ni, n_out, kernel_size=1, stride=1))

    def forward(self, x):
        s = self.low_lvl_conv(sum(self.hook.stored))
        ssh = s.shape[-3:]
        if ssh != x.shape[-3:]:
            x = F.interpolate(x, size=ssh, mode='nearest')
        x = torch.cat((x, s), dim=1)
        return self.last_conv(x)


In [ ]:
# export
class DeepLabClassifier(Module):
    "Decoder Block for DynamicDeeplab"
    def __init__(self, ni, n_out, hook, norm_type=None, 
                 act_cls=defaults.activation, ps=0.25):
        self.hook = hook
        self.pool = AdaptiveConcatPool3d(1)
        self.fc = LinBnDrop(ni*2, n_out, act = act_cls(), p = ps)

    def forward(self, x):
        s = self.pool(sum(self.hook.stored))
        x.class_pred = self.fc(s.flatten(1))
        return x


In [ ]:
# export
class DynamicDeepLab(SequentialExForDeepLab):
    "Build DeepLab with different encoders"
    def __init__(self, encoder, n_out, img_size, y_range=None, add_classifier=False, n_out_classifier=2,
                       act_cls=defaults.activation, norm_type=NormType.Batch, **kwargs):
        
        sizes = model_sizes(encoder, size=img_size)
        sz_chg_idxs = list(_get_sz_change_idxs(sizes))        
        self.sfs = [hook_outputs(encoder[sz_chg_idxs[1]], detach=False)]
        if add_classifier: 
            self.sfs.append(hook_outputs(encoder[len(encoder)-1], detach=False))
    
        x = dummy_eval(encoder, img_size).detach()
        ni = sizes[-1][1]
        nf = ni//4
        dilations=[1, 12, 24, 36] if ni > 512 else [1, 6, 12, 18]
        aspp = ASPP(ni=ni, nf=nf, dilations=dilations, norm_type=norm_type, act_cls=act_cls).eval()
        x = aspp(x)
        decoder = DeepLabDecoder(ni=nf, low_lvl_ni=sizes[sz_chg_idxs[1]][1], hook=self.sfs[0], n_out=n_out, 
                                 norm_type=norm_type, act_cls=act_cls).eval()
        x = decoder(x)
                   
        self.layers = nn.ModuleList([encoder, aspp, decoder, ResizeToOrig()])
        
        if add_classifier: 
            assert n_out_classifier, 'Number of classes need to be specified'
            self.layers.append(DeepLabClassifier(ni, n_out_classifier, self.sfs[1]))


    def __del__(self):
        if hasattr(self, "sfs"): self.sfs.remove()        

In [ ]:
from torchvision.models.video import r3d_18

In [ ]:
body_3d = create_body(r3d_18, pretrained = False)

In [ ]:
m = DynamicDeepLab(body_3d, 2, (20, 64, 64), add_classifier=True)

In [ ]:
a,b = m(torch.randn(2,3,20,64,64))
a.shape, b.shape

(torch.Size([2, 2, 20, 64, 64]), torch.Size([2, 2]))

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 01_basics.ipynb.
Converted 02_preprocessing.ipynb.
Converted 03_transforms.ipynb.
Converted 04_dataloaders.ipynb.
Converted 05_layers.ipynb.
Converted 06_learner.ipynb.
Converted 06a_models.alexnet.ipynb.
Converted 06b_models.resnet.ipynb.
Converted 06c_model.efficientnet.ipynb.
Converted 06d_models.unet.ipynb.
Converted 06e_models.deeplabv3.ipynb.
Converted 06f_models.losses.ipynb.
Converted 07_callback.ipynb.
Converted index.ipynb.
